### L'outil va déterminer quel est le genre de musique entre 
### le Rock, l'Hip Hop et le Reggae 
### à partir des paroles qu'on lui donne en paramètre



In [116]:
import pandas as pd
import numpy as np
import os as os
import re as re
import nltk as nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text      
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

Cet outil correspond à de la Classification de Texte, on va utiliser des modèles d'apprentissage machine supervisé.

Les données qu'on veut sont les paroles de musique et leur genre. Pour utiliser nos modèles on va représenter nos paroles avec la représentation bag of word. On utilisera les modèles grâce à la librairie scikit-learn. Nous utiliserons à la fin de ce notebook des mesures pour evaluer nos modèles et en fonction des résultat exprimer s'il existe des différences notables entre ces 3 genres de musique à partir uniquement de leur parole et s'il est possible de déterminer le genre d'une musique uniquement à partir de ses paroles. On ne traitera que les musiques avec des paroles en anglais !

Les bases de données que nous utiliserons  : [150K Lyrics Labeled with Spotify Valence](https://www.kaggle.com/edenbd/150k-lyrics-labeled-with-spotify-valence?select=labeled_lyrics_cleaned.csv), [Spotify Tracks DB](https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db?select=SpotifyFeatures.csv), [Lyrics Data](https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres?select=lyrics-data.csv). 
On va récuperer dans les paroles des musique Rock et Hip Hop dans la dernière BD et les paroles de musique Reggae dans la première.
En effet le genre reggae n'est pas dans la dernière BD.

On charge nos bases de données dans des dataFrame

In [2]:
path = os.getcwd()
filePathlyrics = path+r'\data\lyrics-data.csv'
filePathLabeledLyrics = path+r'\data\labeled_lyrics_cleaned.csv'
filePathartists = path+r'\data\artists-data.csv'
filePathSpotify = path+r'\data\SpotifyFeatures.csv'
data_lyrics = pd.read_csv(filePathlyrics)
data_artists = pd.read_csv(filePathartists)
data_lyrics_labeled = pd.read_csv(filePathLabeledLyrics)
data_spotify = pd.read_csv(filePathSpotify)

Pour des raisons de mémoires, on va limiter le nombre de musique qu'on veut récuperer et mettre en entrée des modèles

In [3]:
NB_MUSIQUE_MAX = 200
SIZE_TRAIN = 60
SIZE_TEST = 10


Dans la DataFrame `data_lyrics` on s'interesse a :
* La colonne `ALink`, nom de l'artiste (du groupe éventuellement) dans un format d'écriture qui nous servira à faire le lien avec la deuxième DataFrame `data_artist`
* La colonne `Lyric`, les paroles de la musique. S'il y en a pas il est indiqué par le texte 'Instrumental'

Dans la DataFrame `data_artists` on s'interesse a :
* La colonne `Artist`, nom de l'artiste (du groupe éventuellement)
* La colonne `Link`, un format d'écriture du nom de l'artiste
* Le colonne `Genre`, le genre des musiques de l'Artiste 

Dans la DataFrame `data_lyrics_labeled` on s'interesse a :
* La colonne `artist`, nom de l'artiste (du groupe éventuellement)
* La colonne `seq`, les paroles de la musique.

Dans la DataFrame `data_spotify` on s'interesse a :
* La colonne `artist_name`, nom de l'artiste (du groupe éventuellement)
* Le colonne `Genre`, le genre des musiques de l'Artiste

Pour récuperer les paroles de musique Reggae on va chercher les artistes du genre Reggae dans `data_spotify` et on va filtrer les données de `data_lyrics_labeled` pour n'avoir que les artistes du genres Reggae et donc que les paroles du genre Reggea.


L'objectif étant de regrouper dans une DataFrame l'artiste, le genre et les paroles. (On se servira que des 2 dernières pour nos modèles, l'artiste ce n'est que par curiosité :) )

Ci-dessous une visualisation des DataFrame

In [4]:
data_lyrics.head(2)

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH


In [5]:
data_artists.head(2)

,Artist,Songs,Popularity,Link,Genre,Genres
0,10000 Maniacs,110,0.3,/10000-maniacs/,Rock,Rock; Pop; Electronica; Dance; J-Pop/J-Rock; G...
1,12 Stones,75,0.3,/12-stones/,Rock,Rock; Gospel/Religioso; Hard Rock; Grunge; Roc...


In [7]:
data_spotify.head(2)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.0,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.0,F#,0.151,-5.559,Minor,0.0868,174.003,4/4,0.816


In [8]:
data_lyrics_labeled.head(2)

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630


On prépare une fonction de pré-traitement du texte de nos chansons.

Pour cela, on a besoin de :
* Supprimer les caractères qui ne sont pas du texe (des mots)
* Supprimer les "stop words"
* Mettre tous les caractères en miniscule.

Nous traitons les chansons anglaises, on peut alors importer les stop words anglais uniquement avec nltk.
Nous verrons ensuite que certaint pré-traitement ici seront inutiles puisque nous utiliserons les CountVector de sklearn qui convertira en matrice de token les paroles (Lyric). Le module CountVector nous permet de passer en paramètre le langage des stop words, l'encodage etc.


In [108]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_word = stopwords.words("english")
def text_prepoc(txt):
    #supprime les majuscules, la ponctuation : '!?.,' etc
    t = txt.lower() #plus besoin
    #t = ' '.join([word for word in t.split(' ') if word not in stop_word]) # plus besoin avec CountVector
    t = txt.encode('ascii', 'ignore').decode()
    t = re.sub('[^a-zA-Z0-9 ]', ' ', t)
    return t


On veut dans une dataFrame les musiques de hip hop uniquement.

Pour cela on récupère les artistes de Hip Hop dans notre dataFrame `data_artists` grace à la colonne `'Genre'`.
On peut alors, avec la méthode `merge` de Pandas filtrer et récuperer les chanteurs de hip hop qui correspondent dans la dataFrame des paroles. On récupère enfin les colonnes qui nous interesse comme décrites plus haut.

In [10]:
hiphop_artist = data_artists[data_artists['Genre'] == "Hip Hop"]
hiphop_musique = pd.merge(data_lyrics, hiphop_artist, how='inner', left_on='ALink', right_on='Link')
hiphop_musique = hiphop_musique[['Genre', 'Artist', 'SName', 'Lyric', 'Idiom']].rename(columns={'SName':'Titre'})

On s'interresse uniquement aux chansons anglaises. 

(On supprime les chansons sans paroles caratérisées par la valeur `Instrumental` dans la colonne `'Lyric'`)

In [11]:
mask_instrumental = hiphop_musique['Lyric'] != 'Instrumental'
mask_langage = hiphop_musique['Idiom'] == 'ENGLISH'
hiphop_musique = hiphop_musique[mask_instrumental]
hiphop_musique = hiphop_musique[mask_langage]

On limite le nombre de musique par genre à `NB_MUSIQUE_MAX`

In [104]:
hiphop_musique = hiphop_musique.head(NB_MUSIQUE_MAX)
hiphop_musique.applymap(text_prepoc)

,Genre,Artist,Titre,Lyric,Idiom
0,Hip Hop,Beastie Boys,Sabotage,I can t stand it I know you planned it I m a...,ENGLISH
1,Hip Hop,Beastie Boys,Intergalactic,Intergalactic Planetary Planetary Intergalacti...,ENGLISH
2,Hip Hop,Beastie Boys,You Gotta Fight For Your Right To Party,Kick it Verse 1 You wake up late for scho...,ENGLISH
3,Hip Hop,Beastie Boys,Sure Shot,You can t you won t and you don t stop Mike ...,ENGLISH
4,Hip Hop,Beastie Boys,No Sleep Till Brooklyn,chorus No sleep til Brooklyn Foot on t...,ENGLISH
...,...,...,...,...,...
227,Hip Hop,Beastie Boys,Rhymin And Stealin,Because mutiny on the bounty s what we re all ...,ENGLISH
228,Hip Hop,Beastie Boys,3 The Hard Way,Fresh dressed cause I shop at Models Deep in...,ENGLISH
230,Hip Hop,Beastie Boys,Shake Your Rump,Now Randy listen I rock a house party at the d...,ENGLISH
231,Hip Hop,Beastie Boys,Three Mc s And One Dj,Cause nobody can do it like Mix Master Mike ca...,ENGLISH


### On procède de la même manière pour le genre Rock

In [13]:
rock_artist = data_artists[data_artists['Genre'] == "Rock"]
rock_musique = pd.merge(data_lyrics, rock_artist, how='inner', left_on='ALink', right_on='Link')
rock_musique = rock_musique[['Genre', 'Artist', 'SName', 'Lyric', 'Idiom']].rename(columns={'SName':'Titre'})
mask_instrumental = rock_musique['Lyric'] != 'Instrumental'
mask_langage = rock_musique['Idiom'] == 'ENGLISH'
rock_musique = rock_musique[mask_instrumental]
rock_musique = rock_musique[mask_langage]
rock_musique = rock_musique.head(NB_MUSIQUE_MAX)
rock_musique.applymap(text_prepoc)

,Genre,Artist,Titre,Lyric,Idiom
0,rock,10000 maniacs,,could feel time way knowing fallen leaves ni...,english
1,rock,10000 maniacs,night,take now baby am hold close try understand...,english
2,rock,10000 maniacs,days,are days remember never never since promise...,english
3,rock,10000 maniacs,campfire song,lie say o mountain coal veins beds dig 500...,english
4,rock,10000 maniacs,everyday like sunday,trudging slowly wet sand back bench clothes s...,english
...,...,...,...,...,...
196,rock,12 stones,lifeless,desperate waiting frozen core numb feeling ...,english
197,rock,12 stones,little eyes,little eyes see again know sin try hide awa...,english
198,rock,12 stones,memphis,pretend world s changing believe deserve much...,english
199,rock,12 stones,life,find question again doubt love given me hope...,english


Pour le genre Reggae, on va récuperer les paroles comme expliqué plus haut : 
* On extrait de la dataFrame les artistes du genre Reggae uniquement 
* On recupere l'intersection des dataFrame `data_lyrics_labeled` et `reggae_artists` pour obtenir les musiques Reggae dans `data_lyrics_labeled` (l'opération innerjoin)
* On récupère les colonnes qui nous interesse et renome les colonnes pour qu'elles aient les même noms que celles des dataFrame `hiphop_musique` et `rock_musique`


In [ ]:
reggae_artists = data_spotify[data_spotify['genre']=='Reggae'][['genre','artist_name']].drop_duplicates() #Pour supprimer les répetitions
reggae_musique = pd.merge(data_lyrics_labeled, reggae_artists, how='inner', left_on='artist', right_on='artist_name')
reggae_musique = reggae_musique[['genre', 'artist', 'song', 'seq']].rename(columns={'genre':'Genre', 'artist':'Artist', 'song':'Titre','seq':'Lyric'})
reggae_musique = reggae_musique.dropna().head(NB_MUSIQUE_MAX) #drop na values (au cas où...)
reggae_musique['Lyric'] = reggae_musique['Lyric'].apply(text_prepoc)

On stock dans la variable X le tableau final avec les 3 genres et les colonnes qui nous intéresse.
L'idée est d'utiliser un ou plusieurs modèles (Machine Learning) de Classification.
On va découper nos données en données d'entrainement pour les modèles et en données de test pour l'évaluation de ces derniers.
On aura donc les ensembles suivant :
* Les Paroles d'entrainement
* Les Genres qui correspondent
* Les paroles de test
* Les Genres de ces paroles de test

Grâce au module `CountVectorizer` on va représenter nos Lyrics en matrice (Bag of Word representation)

In [107]:
X = pd.concat([hiphop_musique.head(SIZE_TRAIN), rock_musique.head(SIZE_TRAIN), reggae_musique.head(SIZE_TRAIN)])
X

,Genre,Artist,Titre,Lyric,Idiom
0,Hip Hop,Beastie Boys,Sabotage,"I can't stand it, I know you planned it. I'm a...",ENGLISH
1,Hip Hop,Beastie Boys,Intergalactic,Intergalactic Planetary Planetary Intergalacti...,ENGLISH
2,Hip Hop,Beastie Boys,(You Gotta) Fight For Your Right (To Party!),Kick it!. [Verse 1]. You wake up late for scho...,ENGLISH
3,Hip Hop,Beastie Boys,Sure Shot,"You can't, you won't and you don't stop. Mike ...",ENGLISH
4,Hip Hop,Beastie Boys,No Sleep 'Till Brooklyn,(chorus) No sleep 'til - Brooklyn. . Foot on t...,ENGLISH
...,...,...,...,...,...
55,Reggae,Rebelution,Moonlight,"We've been lost out here for days,\r\nDon't wo...",NaN
56,Reggae,Rebelution,More Than Ever,"Just like I said before,\r\nGive me two weeks,...",NaN
57,Reggae,Rebelution,Wake Up Call,"It's a shame when somebody shows,\r\nDiamonds ...",NaN
58,Reggae,Rebelution,Closer I Get [Acoustic],People say it just doesn't matter but\r\nThe c...,NaN


In [66]:
#y = X['Genre']
#y.replace(['Hip Hop', 'Rock', "Reggae"], [1, 2, 3], inplace=True)

In [67]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [70]:
# Utilisation de CountVectorizer de la librairie sklearn
# Convertit nos lyrics en matrice de token
count_vectorizer = CountVectorizer(strip_accents='ascii', lowercase=True, stop_words='english', analyzer='word')
# representation en bag of word
bow = count_vectorizer.fit_transform(X['Lyric'].values)

In [20]:
#Tfidf_vect = TfidfVectorizer(max_features=5000)
#Tfidf_vect.fit(X['Lyric'])
#Train_X_Tfidf = Tfidf_vect.transform(X_train)
#Test_X_Tfidf = Tfidf_vect.transform(X_test)

#SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
#SVM.fit(Train_X_Tfidf,y_train)
#predictions_SVM = SVM.predict(Test_X_Tfidf)

On prépare nos ensembles de test qui va servir à évaluer les modèles

In [71]:
test_data=pd.concat([hiphop_musique.iloc[SIZE_TRAIN:NB_MUSIQUE_MAX+SIZE_TEST], rock_musique.iloc[SIZE_TRAIN:NB_MUSIQUE_MAX+SIZE_TEST],reggae_musique.iloc[SIZE_TRAIN:NB_MUSIQUE_MAX+SIZE_TEST]])
test_bow = count_vectorizer.transform(test_data['Lyric'].values) 

On voulais initialement utiliser les SVM mais on pas réussis à faire fonctionner ce modèle. Même en utilisant une representation Tf-Idf
On cependant tester plusieurs modèles qui ont fonctionné telle que la Régression Linéaire, l'algorithme du Gradient et de l'arbre de décision.

In [78]:
modeles = {'Logistic Regression':LogisticRegression(max_iter=500), 'Linear SVC':svm.LinearSVC(max_iter=10000),'Decision Tree': DecisionTreeClassifier(), 'Gradient Descent': SGDClassifier()}
for model in modeles.keys():
    modeles[model].fit(bow.toarray(), X['Genre'])


In [118]:
accuracy = {}
genre_predicted = test_data[['Artist', 'Titre','Genre']]
for model in modeles.keys():
    pred_genre= modeles[model].predict(test_bow.toarray())
    genre_predicted[model]=pred_genre
    accuracy[model]=accuracy_score(test_data['Genre'],pred_genre)
    print(f"Report du modèle {model} :\n", classification_report(test_data['Genre'], pred_genre))

Report du modèle Logistic Regression :
               precision    recall  f1-score   support

     Hip Hop       0.70      0.87      0.77       140
      Reggae       0.59      0.30      0.40       140
        Rock       0.63      0.78      0.69       140

    accuracy                           0.65       420
   macro avg       0.64      0.65      0.62       420
weighted avg       0.64      0.65      0.62       420

Report du modèle Linear SVC :
               precision    recall  f1-score   support

     Hip Hop       0.67      0.81      0.73       140
      Reggae       0.55      0.34      0.42       140
        Rock       0.63      0.74      0.68       140

    accuracy                           0.63       420
   macro avg       0.62      0.63      0.61       420
weighted avg       0.62      0.63      0.61       420

Report du modèle Decision Tree :
               precision    recall  f1-score   support

     Hip Hop       0.66      0.76      0.71       140
      Reggae       0.57 

In [82]:
for model, score in accuracy.items():
    print(f'Le modèle {model} à une précision de predictions de {round(score*100,2)}%')

Le modèle Logistic Regression à une précision de predictions de 65.0%
Le modèle Linear SVC à une précision de predictions de 63.1%
Le modèle Decision Tree à une précision de predictions de 62.14%
Le modèle Gradient Descent à une précision de predictions de 56.19%


On voit que le modèle avec le meilleur score est la régression linéaire, en se fiant au classification report plus haut.
Mais les scores restent néanmoins faibles ~60%. Cela peut-être dû à la taille de nos données en entrée des modèles qui sont limités volontairement pour des raisons de mémoire. 

Ce taux d'erreur élevé peut aussi signifier que le vocabulaire n'est pas le critère determinant le genre d'une musique, il y a surtout le son, "l'instrumental" qui determine essentiellement le genre auquelle elle va appartenir. On a volontairement choisi de prendre des genres très différents parce que la tâche de reconnaître le genre entre des styles proches à partir des paroles uniquements est quasi-impossible, si on prends une musique metal et une hard rock, il faut se tourner aux sons pour les différencier.

Pour améliorer notre outil, on peut essayer d'utiliser d'autres modèles et d'autres representation des mots. Analyser le sons mais dans ce cas on sort du cadre du cours. On peut aussi et c'est une amélioration très intéressante faire un génerateur de paroles en fonction du genre qu'on souhaite qui pourrait se baser sur les prédictions de cet outil.

In [87]:
genre_predicted

,Artist,Titre,Genre,Logistic Regression,Linear SVC,Decision Tree,Gradient Descent
70,Beastie Boys,Dope Little Song,Hip Hop,Hip Hop,Hip Hop,Reggae,Hip Hop
71,Beastie Boys,"Dr. Lee, Phd",Hip Hop,Rock,Rock,Rock,Rock
74,Beastie Boys,Drunken Praying Mantis Style,Hip Hop,Hip Hop,Hip Hop,Hip Hop,Hip Hop
75,Beastie Boys,Dub The Mic,Hip Hop,Hip Hop,Hip Hop,Hip Hop,Hip Hop
76,Beastie Boys,Egg Raid On Mojo,Hip Hop,Hip Hop,Reggae,Rock,Reggae
...,...,...,...,...,...,...,...
195,UB40,The Train Is Coming,Reggae,Reggae,Reggae,Reggae,Reggae
196,UB40,The Earth Dies Screaming,Reggae,Rock,Rock,Rock,Rock
197,UB40,Cover Up,Reggae,Rock,Rock,Reggae,Rock
198,UB40,Rudie,Reggae,Rock,Rock,Rock,Reggae


In [123]:
mes_paroles = input()
bow_mes_paroles = count_vectorizer.transform([text_prepoc(mes_paroles)]) 
for model in modeles.keys():
    p = modeles[model].predict(bow_mes_paroles)
    print(f"Le modèle {model} pense que vos parole se rapproche du genre {p[0]}")

Le modèle Logistic Regression pense que vos parole se rapproche du genre Hip Hop
Le modèle Linear SVC pense que vos parole se rapproche du genre Hip Hop
Le modèle Decision Tree pense que vos parole se rapproche du genre Rock
Le modèle Gradient Descent pense que vos parole se rapproche du genre Rock
